<a href="https://colab.research.google.com/github/Jieunlee-svg/automation/blob/main/%5B%EB%B6%84%EC%84%9D%5D_dev_noti_wellcheck_error_2%EC%B0%A8%EC%8B%9C%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 크롬 설치
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt -f install -y -qq

--2025-09-24 08:36:01--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 74.125.26.93, 74.125.26.136, 74.125.26.190, ...
Connecting to dl.google.com (dl.google.com)|74.125.26.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120353168 (115M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 114.78M   297MB/s    in 0.4s    

2025-09-24 08:36:02 (297 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [120353168/120353168]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (140.0.7339.207-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libvu

In [2]:
# 크롬 드라이버 설치 - 버전 확인 및 설치
!CHROME_VERSION=$(google-chrome --version | grep -oP '\d+\.\d+\.\d+') && \
  DRIVER_VERSION=$(curl -s "https://googlechromelabs.github.io/chrome-for-testing/LATEST_RELEASE_${CHROME_VERSION}") && \
  wget -q -O chromedriver.zip https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/${DRIVER_VERSION}/linux64/chromedriver-linux64.zip && \
  unzip chromedriver.zip && \
  mv chromedriver-linux64/chromedriver /usr/bin/chromedriver && \
  chmod +x /usr/bin/chromedriver

Archive:  chromedriver.zip
  inflating: chromedriver-linux64/LICENSE.chromedriver  
  inflating: chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: chromedriver-linux64/chromedriver  


In [3]:
# 셀레니움 설치
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = '/usr/bin/google-chrome'  # Colab 환경에서는 필수!

In [5]:
# 드라이버 실행
service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(service=service, options=chrome_options)

In [6]:
# 정상 작동 테스트
driver.get("https://www.google.com")
print("✅ 페이지 타이틀:", driver.title)

✅ 페이지 타이틀: Google


In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 1. 로그인 페이지 이동
driver.get("https://admin.well-check.co.kr/login")

# 2. 입력창이 렌더링될 때까지 대기
wait = WebDriverWait(driver, 10)
user_input = wait.until(EC.presence_of_element_located((By.NAME, "userId")))
pass_input = wait.until(EC.presence_of_element_located((By.NAME, "passwd")))

# 3. 아이디/비밀번호 입력
user_input.send_keys("jel@mcircle.biz")
pass_input.send_keys("Pw@01604")

# 4. 로그인 버튼 클릭
login_btn = driver.find_element(By.TAG_NAME, "button")
login_btn.click()

# 5. 로그인 완료 후 대기
time.sleep(3)

# 확인
print("✅ 로그인 완료, 현재 페이지:", driver.title)

✅ 로그인 완료, 현재 페이지: Home :: 웰체크 어드민 ::


In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1. 고객 테이블 페이지로 이동
driver.get("https://admin.well-check.co.kr/users/customers")

In [10]:
!pip install gspread oauth2client

In [11]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# 스프레드시트 열기
spreadsheet = gc.open("[분석] dev-noti-wellcheck-error")
worksheet = spreadsheet.sheet1

In [12]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# C열: 휴대폰 번호 / F열: 가입일 기록
phone_numbers = worksheet.col_values(3)  # 3번째 열

for i, phone in enumerate(phone_numbers[1:], start=2):  # 2행부터 시작
    if not phone.strip():
        continue

    # 고객 페이지로 매번 새로 이동
    driver.get("https://admin.well-check.co.kr/users/customers")

    # 검색창 찾기 (검색 input의 selector 확인 필요)
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='text']"))
    )
    search_box.clear()
    search_box.send_keys(phone)
    search_box.send_keys(Keys.RETURN)

    time.sleep(2)  # 검색 결과 로딩 대기

    try:
        # 가입일 추출: 테이블 첫 행의 5번째 열
        signup_date_elem = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((
                By.CSS_SELECTOR,
                "table tbody tr:nth-child(1) td:nth-child(5)"
            ))
        )
        signup_date = signup_date_elem.text
    except:
        signup_date = "NOT FOUND"

    # F열에 기록
    worksheet.update_cell(i, 6, signup_date)
    print(f"{phone} → {signup_date}")


821038049867 → 2022.02.07 02:31
821062195705 → 2025.05.03 01:27
821045387624 → 2022.10.06 00:51
821027616031 → 2022.05.24 05:29
821090610021 → 2025.07.30 02:19
821032873365 → 2022.07.09 01:36
821074492400 → 2025.08.06 03:19
821041014190 → 2025.08.07 05:40
821041915949 → 2025.01.20 04:01
821099470273 → 2023.12.21 06:28
821066762876 → 2024.07.23 03:36
821066219580 → 2024.05.28 07:45
821077558577 → 2022.09.02 07:56
821066230375 → 2024.10.31 05:24
821044787301 → 2024.05.10 23:13
821099470273 → 2023.12.21 06:28
821030276358 → 2022.09.30 00:11
821063583273 → 2024.02.16 02:33
821032864223 → 2024.07.25 08:44
821054118223 → 2024.01.15 05:20
821066219580 → 2024.05.28 07:45
821090274343 → 2024.08.05 08:33
821089568673 → 2024.07.06 01:26
821090274343 → 2024.08.05 08:33
821098870282 → 2022.05.17 23:55
821042407286 → 2024.11.27 01:37
821052835325 → 2022.01.15 06:36
821054112638 → 2025.02.24 04:36
821072987198 → 2024.11.09 01:17
821042269272 → 2022.02.26 03:09
821036121138 → 2025.08.06 01:00
82104857